<a href="https://colab.research.google.com/github/davidmorrison-08-30/Cranfield_experiment/blob/main/cranfield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Thông tin nhóm:



*   Nguyễn Nguyên Khôi - 21521009
*   Bùi Quốc Thịnh - 20520934



# Các libraries cần thiết

In [1]:
!git clone https://github.com/samujjwaal/Cranfield-Vector-Space-Model.git

Cloning into 'Cranfield-Vector-Space-Model'...
remote: Enumerating objects: 4768, done.
remote: Counting objects: 100% (4768/4768), done.
remote: Compressing objects: 100% (4471/4471), done.
remote: Total 4768 (delta 1939), reused 3120 (delta 294), pack-reused 0
Receiving objects: 100% (4768/4768), 1.52 MiB | 4.65 MiB/s, done.
Resolving deltas: 100% (1939/1939), done.


In [4]:
# Importing dependancy libraries
import os
import pandas as pd
import numpy as np
import re
import math as m
from collections import Counter
from bs4 import BeautifulSoup
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

In [5]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
stop_list = stopwords.words('english')
# sorted(stop_list)

In [27]:
# Declaring variables for file path
in_path = 'me/cranfieldDocs'
out_path = 'me/preprocessed_cranfieldDocs'

# Declaring variables for query files
query = 'me/queries.txt'
preproc_query = 'me/preprocessed_queries.txt'

# Declaring variable for file with query relevance values
relevance = 'me/relevance.txt'

# Checking if the preprocessed docs folder exists already
if not os.path.isdir(out_path):
    os.mkdir(out_path)

# Getting all filenames from the docs folder
filenames = os.listdir(in_path)  # To generate file path

# Initiallizing Porter Stemmer object
st = PorterStemmer()

# Initializing regex to remove words with one or two characters length
shortword = re.compile(r'\W*\b\w{1,2}\b')

In [28]:
len(filenames)

1400

In [29]:
filenames.sort()
filenames

['cranfield0001',
 'cranfield0002',
 'cranfield0003',
 'cranfield0004',
 'cranfield0005',
 'cranfield0006',
 'cranfield0007',
 'cranfield0008',
 'cranfield0009',
 'cranfield0010',
 'cranfield0011',
 'cranfield0012',
 'cranfield0013',
 'cranfield0014',
 'cranfield0015',
 'cranfield0016',
 'cranfield0017',
 'cranfield0018',
 'cranfield0019',
 'cranfield0020',
 'cranfield0021',
 'cranfield0022',
 'cranfield0023',
 'cranfield0024',
 'cranfield0025',
 'cranfield0026',
 'cranfield0027',
 'cranfield0028',
 'cranfield0029',
 'cranfield0030',
 'cranfield0031',
 'cranfield0032',
 'cranfield0033',
 'cranfield0034',
 'cranfield0035',
 'cranfield0036',
 'cranfield0037',
 'cranfield0038',
 'cranfield0039',
 'cranfield0040',
 'cranfield0041',
 'cranfield0042',
 'cranfield0043',
 'cranfield0044',
 'cranfield0045',
 'cranfield0046',
 'cranfield0047',
 'cranfield0048',
 'cranfield0049',
 'cranfield0050',
 'cranfield0051',
 'cranfield0052',
 'cranfield0053',
 'cranfield0054',
 'cranfield0055',
 'cranfiel

## Preprocessing the documents and queries file

In [30]:
def tokenize(data):
    """Preprocesses the string given as input. Converts to lower case,
    removes the punctuations and numbers, splits on whitespaces, 
    removes stopwords, performs stemming & removes words with one or 
    two characters length.

    Arguments:
        data {string} -- string to be tokenized

    Returns:
        string -- string of tokens generated
    """

    # converting to lower case
    lines = data.lower()

    # removing punctuations by using regular expression
    lines = re.sub('[^A-Za-z]+', ' ', lines)

    # splitting on whitespaces to generate tokens
    tokens = lines.split()

    # removing stop words from the tokens
    clean_tokens = [word for word in tokens if word not in stop_list]

    # stemming the tokens
    stem_tokens = [st.stem(word) for word in clean_tokens]

    # checking for stopwords again
    clean_stem_tokens = [word for word in stem_tokens if word not in stop_list]

    # converting list of tokens to string
    clean_stem_tokens = ' '.join(map(str,  clean_stem_tokens))

    # removing tokens with one or two characters length
    clean_stem_tokens = shortword.sub('', clean_stem_tokens)

    return clean_stem_tokens

In [31]:
def extractTokens(beautSoup, tag):
    """Extract tokens of the text between a specific SGML <tag>. The function
    calls tokenize() function to generate tokens from the text.

    Arguments:
        beautSoup {bs4.BeautifulSoup} -- soup bs object formed using text of a file
        tag {string} -- target SGML <tag>

    Returns:
        string -- string of tokens extracted from text between the target SGML <tag>
    """

    # extract text of a particular SGML <tag>
    textData = beautSoup.findAll(tag)

    # converting to string
    textData = ''.join(map(str, textData))
    # remove the SGML <tag> from text
    textData = textData.replace(tag, '')

    # calling function to generate tokens from text
    textData = tokenize(textData)

    return textData

In [32]:
# Preprocessing all the documents in the cranfieldDocs directory

"""This cell might take about 20 seconds to run."""

for fname in filenames:

    # generate filenames
    infilepath = in_path + '/' + fname
    outfilepath = out_path + '/' + fname

    with open(infilepath) as infile:
        with open(outfilepath, 'w') as outfile:

            # read all text in a file
            fileData = infile.read()

            # creating BeautifulSoup object to extract text between SGML tags
            soup = BeautifulSoup(fileData)

            # extract tokens for <title>
            title = extractTokens(soup, 'title')

            # extract tokens for <text>
            text = extractTokens(soup, 'text')

            # write tokens for <title> into new file
            outfile.write(title)
            outfile.write(" ")

            # write tokens for <text> into new file
            outfile.write(text)
        outfile.close()
    infile.close()

In [33]:
# Preprocessing the queries.txt file
q = open(query)

# opening new file to write preprocessed tokens into
new_q = open(preproc_query, 'w')

# read each line of file seperately
text = q.readlines()
for line in text:
    
    # if condition to avoid newline character in the end of file
    if(line != text[-1]):
        query_tokens = tokenize(line)
        new_q.write(query_tokens + '\n')
    else:
        query_tokens = tokenize(line)
        new_q.write(query_tokens)

In [34]:
# Generate a single list of all preprocessed docs
all_docs = []

for fname in filenames:
    outfilepath = out_path + '/' + fname
    with open(outfilepath) as file:
        fileData = file.read()
        
        # append file data to the list
        all_docs.append(fileData)
# print(all_docs)


In [35]:
# total number of documents is 1400
no_of_docs = len(all_docs)
no_of_docs

1400

# Câu 1: Thống kê các từ được sử dụng trong tập tài liệu Cranfield để biết có bao nhiêu mục từ, mỗi mục từ xuất hiện bao nhiêu lần và xuất hiện trong bao nhiêu tài liệu

In [36]:
DF = {}

for i in range(no_of_docs):
    tokens = all_docs[i].split()
    for w in tokens:
        try:
            # add token as key and doc number as value is chained
            DF[w].add(i)
        except:
            # to handle when a new token is encountered
            DF[w] = {i}

for i in DF:
    # convert to number of occurences of the token from list of documents where token occurs
    DF[i] = len(DF[i])

In [37]:
# tạo tập từ vựng
vocab = [term for term in DF]

In [38]:
# đếm số mục từ có trong tập từ vựng
vocab_size = len(vocab)
print("Số mục từ trong tập từ vựng:", vocab_size)

Số mục từ trong tập từ vựng: 4394


In [39]:
# lưu kết quả câu 1 vào một file .txt
with open("vocab_stats.txt", "w") as f:
  for i in DF.keys():
    data = i + "\t" + str(DF[i]) + "\n"
    f.write(data)

# Câu 3: Lập chỉ mục và tìm kiếm theo mô hình vector

## Lập chỉ mục

In [40]:
# initiate a dictionary acting as an indexing
indexing = {word: [] for word in vocab}

In [41]:
for i in range(no_of_docs):
    
    tokens = all_docs[i].split()
    
    # counter object efficiently count number of occurence of a term in a particular document
    counter = dict(Counter(tokens))
    for token in counter.keys():
      tok_info = [i, counter[token]]
      indexing[token].append(tok_info) 
    words_count = len(tokens)

In [42]:
# cột "Từ khóa"
words = []
for i in vocab:
  words.extend(len(indexing[i])*[i])

In [43]:
# cột "Chỉ số tài liệu"
doc_ind = []
for i in indexing.values():
  doc_ind.extend([k[0] for k in i])  

In [44]:
# cột "Tần số"
doc_freq = []
for i in indexing.values():
  doc_freq.extend([k[1] for k in i])

In [45]:
indexing_pd = pd.DataFrame({"Từ khóa": words, "Chỉ số tài liệu": doc_ind, "Tần số": doc_freq})
indexing_pd.head(7)

,Từ khóa,Chỉ số tài liệu,Tần số
0,experiment,0,2
1,experiment,10,1
2,experiment,11,1
3,experiment,15,1
4,experiment,16,1
5,experiment,18,1
6,experiment,24,1


In [46]:
# lưu xuống file .csv
indexing_pd.to_csv("Chỉ mục.csv")